# Deep Learning with TensorFlow Module 3

[Based on this CognitiveClass Course](https://cognitiveclass.ai/courses/deep-learning-tensorflow/)

## Recurrent Neural Nets

### Sequential Data

Sequential data produces a problem when trying to make use of traditional feed-forward neural networks

NNs assume that inputs are independent and therefore do not take into consideration the impact of previous data

### Recurrent Neural Networks

RNNs are able to maintain a state or context which allows the model to know what was previously calculated. The model feeds back in previous inputs with the new input in order to help us predict sequential information


$$
h_{new}=tanh(W_h\cdot h_{prev}+W_x\cdot x)
$$

Where $h_{prev}$ is the context, and the new state $h_{new}$ which will yield an output $y$

$$
y=W_y\cdot h_{new}
$$

RNNs need to keep track of state at any given timepoint and can become computationally expensive. These are also very sensative to changes in parameters and can suffer from a Vanishing or Exploding gradient

Overall these models can be more difficult to train

### Long Short-Term Memory

A standard RNN can be difficult when trying to learn very long sequences

A solution to this is the LSTM model which maintains a strong gradient over relatively long sequences

An LSTM is contained of a memory cell and 3 logistic gates for writing, reading, and forgeting which control the flow of data

Manipulating these gates allows an LSTM to remember the appropriate level of information

We can also step LSTMs on top of each other to create hierarchial data representations

During the training process the network will learn how much old, and new information to remember as well as the weights and biases based on the different levels of state for each gate at each layer

### Language Modeling

Language modeling is the process of assigning prbabilities to a sequence of words. We can make use of an RNN to form the context based on previous words and thereafter output the new word

Word embedding is a way to process words by decoding a word to a spefici vector. The vectors for the vocabulary are initialized randomly and then are updated based on the context that the word is the same and hence words used in similar contexts 